
# Utility to store views and table definitions in an easily searchable table

1. Define variables and import libraries needed
2. Collect view definition
3. Collect table definition

In [ ]:
dbutils.widgets.text('db_name','next_level_dm')
dbutils.widgets.text('user_table_def_name','dbx_user_tab_def')

In [ ]:
in_db_name = dbutils.widgets.get('db_name')
in_user_tab_def_name = dbutils.widgets.get('user_table_def_name')

In [ ]:
from pyspark.sql.functions import lit
from datetime import datetime

In [ ]:
def user_view_definitions(db_name: str):
    df_views = spark.sql(f"show views in {db_name}" )
    user_view_def = None
    for row in df_views.collect():
        df_view = spark.sql(f"show create table {db_name}.{row.viewName}" )
        df_view = df_view.withColumn('database_name',lit(db_name)).withColumn('table_name',lit(row.viewName)).withColumn('type',lit('view'))
        user_view_def = df_view.alias('user_view_def') if user_view_def is None else user_view_def.unionAll(df_view)
    return user_view_def

In [ ]:
def user_tab_definitions(db_name: str, view_list: []):
    df_tables = spark.sql(f"show tables in {db_name}")
    user_tab_def = None
    for row in df_tables.collect():
        if row.tableName not in view_list:
            df_table = spark.sql(f"show create table {db_name}.{row.tableName}")
            df_table = df_table.withColumn('database_name',lit(db_name)).withColumn('table_name',lit(row.tableName)).withColumn('type',lit('table'))
            user_tab_def = df_table.alias('user_tab_def') if user_tab_def is None else user_tab_def.unionAll(df_table)
    return user_tab_def

In [ ]:
def df_rows_to_list(df, column_name) -> []:
    if df is None:
        return []
    else:
        return [t[0] for t in df.select(column_name).toLocalIterator()]

Prepare view and table definitions

In [ ]:
user_view_def = user_view_definitions(in_db_name)
user_tab_def = user_tab_definitions(in_db_name, df_rows_to_list(user_view_def,'table_name'))

user_obj_def = user_tab_def if user_view_def is None else user_view_def.unionAll(user_tab_def)
user_obj_def = user_obj_def.withColumn('latest_metadata_refresh', lit(datetime.now()))

Store results to a table

In [ ]:
user_obj_def.write.mode("overwrite").saveAsTable(f"{in_db_name}.{in_user_tab_def_name}")